In [1]:
import numpy as np
import matplotlib.pyplot as plt

SINGLE NEURON 

In [ ]:
def single_neuron(inputs):
    
    l = len(inputs)
    
    weights = np.random.rand(1,l)
    bias    = np.ranom.rand(1,1)
    out     = np.dot(weights,inputs) + bias
    return(out)

LAYER OF NEURONS

ACTIVATION FUNCTION 

SOFTMAX

LOSS AND ENTROPY 

BACKPROP 

OPTIMIZER

LEARNING AND TRAINING 

CONVOLUTION

POOLING

SIGMOID AND TANH

FLATTENING

BACKPROB AGAIN 

TESTING